# Inference example

In [ ]:
import sys
sys.path.append('../')

import cv2
import matplotlib.pyplot as plt

from detectron2 import config
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer

from d2.detr import DetrDatasetMapper, add_detr_config

## Define paths and constants

In [ ]:
CONFIG_PATH = 'logs/coco_finetune/config.yaml'
MODEL_WEIGHTS = 'logs/coco_finetune/model_final.pth'
IMAGE_PATH = '/opt/kuna/data/clean_vertexFull100k_S2018/vertexFull100k_S2018/JPEGImages/KNP04BK001041800206/2018_07_05__12_20_15-0400-0000000-1.JPEG'
# Arbitrary threshold, this one is taken from the official colab, without any data specific tuning
THRESHOLD = 0.7

In [ ]:
cfg = config.get_cfg()
add_detr_config(cfg)
cfg.merge_from_file(CONFIG_PATH)
cfg.MODEL.WEIGHTS = MODEL_WEIGHTS
# Change to cpu here if needed
cfg.MODEL.DEVICE='cuda:0' 

## Create predictor

In [ ]:
predictor = DefaultPredictor(cfg)

In [ ]:
im = cv2.imread(IMAGE_PATH)
outputs = predictor(im)

In [ ]:
instances = outputs['instances']
indices = (outputs['instances'].scores > THRESHOLD).nonzero().squeeze()
if len(indices.size()) > 0:
    instances = instances[indices]
else:
    instances = instances[indices.item()]

v = Visualizer(im[:, :, ::-1])
v = v.draw_instance_predictions(instances.to("cpu"))
plt.figure(figsize = (16,9))
plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))

## Calculate FPS

In [ ]:
import time
import numpy as np

times = []
for i in range(20):
    start_time = time.time()
    outputs = predictor(im)
    delta = time.time() - start_time
    times.append(delta)
mean_delta = np.array(times).mean()
fps = 1 / mean_delta
print("Average(sec):{:.2f}, fps:{:.2f}".format(mean_delta, fps))